# Lecture 2, Part 1: Introduction to Neural Networks

*Credit*: this notebook is based on FastML [tutorials](https://github.com/fastmachinelearning/hls4ml-tutorial/blob/master/part1_getting_started.ipynb).

In [ ]:
# First, let's import torch, numpy and matplotlib

import numpy as np
import torch
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Let's fix the random seed to have reproducable the results. 
# Later you can try changeing the SEED to see how much does it influence the results

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

## Warmup: PyTorch

Wikipedia: *PyTorch is an open source machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing, primarily developed by Meta AI.*

Let's do a few operations to get used to it.

In [ ]:
# Initialize a random tensor
t = torch.rand((4, 4))
print(t)
type(t)

In [ ]:
# Create a vector
a = torch.arange(16)
a

In [ ]:
# Reshape to a matrix
a = a.reshape((4, -1))
a

In [ ]:
# Get a new tensor filled with ones
b = torch.ones_like(a)
b

In [ ]:
# Add tensors 
b + a

In [ ]:
# Multiply tensors
(2*b) * a

In [ ]:
# Matrix product of two tensors
b.matmul(a)

## Introduction

The majority of particles produced in LHC events are unstable and immediately decay to lighter particles. The new particles can decay themselves to others in a so-called decay chain. Such a process terminates when the decay products are stable particles, e.g., charged pions. This collimated shower of particles with adjacent trajectories is called a *jet*. Jets are central to many physics studies at the LHC experiments. In particular, a successful physics program requires aggregating particles into jets (jet clustering), an accurate determination of the jet momentum (momentum measurement) and the identification of which particle kind started the shower (**jet tagging**).

In this excercise you will learn how to train and evaluate a classifier that will predict which particle produced a given jet, a *jet tagger*. To this purpose we will download the dataset containing samples with 16 high-level features from [here](https://paperswithcode.com/dataset/hls4ml-lhc-jet-dataset). We will classify jets to five classes: produced by decay of top (*t*), Z (*z*) and W (*w*) boson, gluon (*g*) and quark (*q*).

<img src='https://production-media.paperswithcode.com/datasets/JetDataset.png' width='300px'>

## Downloading, exploring and preprocessing the dataset

In [ ]:
# We are going to download the dataset now

from sklearn.datasets import fetch_openml

data = fetch_openml('hls4ml_lhc_jets_hlf')
# In colab execute this line: X, y = data['data'].to_numpy(), data['target'].to_numpy()
X, y = data['data'], data['target']

In [ ]:
# Let's explore the dataset

print('We have {} features in the dataset'.format(len(data['feature_names'])))
print('Number of datapoints in the dataset {}'.format(X.shape[0]))
print('Are X and y the same size? {}'.format(X.shape[0]==y.shape[0]))
print('Our target classes for tagging: {}'.format(set(y)))

In [ ]:
print(y[0], X[0])

As you saw above, the **y** target is a string, e.g. 'g'. We need to modify it to a number.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_int = le.fit_transform(y)
print([(x,y) for x,y in zip(le.classes_, range(len(le.classes_)))])
print(y_int[0])

OK, this looks better. We can work with numbers. But wait, this still doesn't make sense. Why the distance between *q* and *t* is shorter than *q* and *w*? It's not really better to classify *q* as *t* rather than *w*, does it? We need to make this a "one hot" which will equalize the errors made by model.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_true = ohe.fit_transform(y_int.reshape(-1, 1))
print(ohe.categories_)
print(y_true[0])

Let's split the dataset. 80:20:20 (train, validation, test) split is usually a good start

In [ ]:
# Train test split

from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_true, test_size=0.2, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=SEED)

print('training:')
print(X_train.shape)
print(y_train.shape)

print('validation:')
print(X_val.shape)
print(y_val.shape)

print('test:')
print(X_test.shape)
print(y_test.shape)

Let's explore the dataset. Visualizing each feature will help us understand what do we use as an input

In [ ]:
# Loop over the features and plot a histogram

for i in range(len(data['feature_names'])):
    plt.hist(X[:,i], bins=50);
    plt.title(data['feature_names'][i])
    plt.show();

### Your task: Try to visualize it again but for each class separatelly

In [ ]:
for i in range(len(data['feature_names'])):
    # TODO: Your code here
    plt.show();

We could notice the differences in the scale of different features. We should normalize the dataset before training our neural network.

In [ ]:
# We scale the input

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## Constructing the model

Now, let's construct a simple model. We'll use 3 hidden layers, each with 32 neurons. Each fully connected layer will be folowed by ReLU activation. The output layer has 5 nodes (one for each class). To get the probability distribution we use the softmax activation.

In [ ]:
# Finally, let's construct the model

import torch.nn.functional as F
from torch.nn import Linear

class JetTagger(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(JetTagger, self).__init__()
        # Let's define our layers here.
        self.fc1 = torch.nn.Linear(in_features, 32, bias=True) # Define the first layer (input: in_features and 32 outputs)
        self.fc2 = torch.nn.Linear(32, 32, bias=True) # Define the second hidden layer 
        self.fc3 = torch.nn.Linear(32, 32, bias=True) # Define the third hidden layer 
        self.fc4 = torch.nn.Linear(32, out_features, bias=True) # Define the output layer 

    def forward(self, x):
        # Let's define out forward pass
        x = F.relu(self.fc1(x)) # Pass through the first layer and relu activation
        x = F.relu(self.fc2(x)) # Now through the second
        x = F.relu(self.fc3(x)) # And third
        x = F.softmax(self.fc4(x), dim=-1) # Our output should be normalized by softmax, i.e. summing to 1.
        return x

### Your task: Pass correct arguments to match number of input and output nodes

In [ ]:
net = JetTagger(_fix_me) #TODO: pass correct arguments

We can check the model simply by printing it:

In [ ]:
print(net)

We can also check the weights of a layer of interest, e.g.:

In [ ]:
print(net.fc2.weight)

In [ ]:
# Maybe you will need to execute this line first:
#!pip install torchviz

In [ ]:
# Let's visualize the computational graph

from torchviz import make_dot

net.eval()
yhat = net(torch.zeros(1, 16))
make_dot(yhat, params=dict(list(net.named_parameters())))

This is a nice visualization of the entire graph. But what if we just want see the architecutre?

There are many tools, so let's try one of them. First we need to export the model to TorchScript.

In [ ]:
model_scripted = torch.jit.script(net) 
model_scripted.save('myJetTagger.pt')

Now go to [netron](https://netron.app/) and click 'Open model'. Select your `myJetTagger.pt` file. It should show something like this:
<img src="./assets/jettagger.png">

## Training the model

PyTorch provides the `Dataset` and `DataLoader` primitives to faciliate data handling, see [here](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). Let's use them!


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 1024

train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In order to monitor our training let's write a helper class that will keep track of accuracy and loss:

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':1.5f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{avg' + self.fmt + '} ({name})'
        return fmtstr.format(**self.__dict__)

We'll use simple SGD optimizer and crossentropy loss ($CE$):
$$ CE=-\sum_{i}^{C} t_i \log s_i$$ where $t_i$ and $s_i$ are the ground truth and network outputs for each class $i$ in $C$.

The model isn't very big, so this should just a moment on a regular CPU. For bigger models you'd like to use GPUs. You can increase number of epochs to see what happens.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.0001)
epochs = 10

Finally, the most crucial part, where the learning happens. Take a moment to read the code to understand each step.

In [ ]:
import sys

from tqdm.notebook import trange

acc, loss = AverageMeter('Accuracy'), AverageMeter('Loss')
train_loss, val_acc, val_loss = [], [], []

# Iterate over the dataset <epochs> times
for epoch in range(epochs):

    # Set the model to training mode
    net.train()
    # Reset our meters
    loss.reset()
    acc.reset()

    # This is just here because it's pretty 
    tr = trange(len(train_dataloader), file=sys.stdout)

    # Iterate over batches
    for inputs, targets in train_dataloader:

        # Remove previous gradients
        optimizer.zero_grad()
        
        # Feed forward the input
        outputs = net(inputs)

        # Compute the loss and accuracy
        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)
        
        preds = torch.argmax(outputs, dim=-1)
        accuracy = (torch.argmax(targets, dim=-1) == preds).sum() / len(targets)
        acc.update(accuracy.data)

        # Show the current results
        tr.set_description('Epoch {}, {}, {}'.format(epoch+1, loss, acc))
        tr.update(1)

        # Compute the gradients
        loss_batch.backward()

        # Update parameters
        optimizer.step()
        
    train_loss.append(loss.avg)
    
    # Validation for each epoch
    net.eval()
    loss.reset()
    acc.reset()

    tr = trange(len(val_dataloader), file=sys.stdout)

    for inputs, targets in val_dataloader:

        outputs = net(inputs)

        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)

        preds = torch.argmax(outputs, dim=-1)
        accuracy = (torch.argmax(targets, dim=-1) == preds).sum() / len(targets)
        acc.update(accuracy.data)

        tr.set_description('Validation, {}, {}'.format(loss, acc))
        tr.update(1)

    val_loss.append(loss.avg)
    val_acc.append(acc.avg)

In [ ]:
# Let's draw loss and accuracy for the training and validation

def draw_loss(data_train, data_val, data_acc, label="Loss"):
    """Plots the training and validation loss"""

    fig, ax1 = plt.subplots()
    ax1.set_xlabel("Epoch", horizontalalignment='right', x=1.0)
    ax1.set_ylabel("Loss", horizontalalignment='right', y=1.0)
    ax1.set_yscale('log')
    ax1.tick_params(axis='y', labelcolor='red')
    ax1.plot(data_train,
             color='red',
             label='Training loss')
    ax1.plot(data_val,
             color='blue',
             label='Validation loss')
    ax2 = ax1.twinx()
    ax2.set_ylabel('Accuracy', color='black')
    ax2.tick_params(axis='y', labelcolor='black')
    ax2.plot(data_acc,
             color='green',
             label='Accuracy')
    ax1.legend(loc='lower left')
    ax2.legend(loc='upper left')
    plt.show()

draw_loss(train_loss, val_loss, val_acc)

Let's check the performance of the model, i.e. accuracy and make a the ROC curve

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = net(torch.tensor(X_test).unsqueeze(0).float())

print("Accuracy for the test set: {0:.2f}".format(
    accuracy_score(
        np.argmax(y_test, axis=1),
        torch.argmax(y_pred, dim=-1).squeeze().numpy())
))

In [ ]:
from sklearn.metrics import roc_curve, auc

def plot_roc(y_test, y_pred, labels):
    for x, label in enumerate(labels):        
        fpr, tpr, _ = roc_curve(y_test[:, x], y_pred[:, x])
        plt.plot(tpr, fpr, label='{0} tagger, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle='-')
    plt.semilogy()
    plt.xlabel("Signal Efficiency")
    plt.ylabel("Background Efficiency")
    plt.ylim(0.001, 1)
    plt.grid(True)
    plt.legend(loc='upper left')  
    
plt.figure(figsize=(9, 9))
plot_roc(y_test, y_pred.squeeze().detach().numpy(), le.classes_)

So this really doesn't work...
### Your task: Try to fix the problem...
Hint: Look at the loss, why is it decreasing so slowly?

### Finshed?

After the simple adjustment you should get at least 70% accuracy. Can you do better? Go ahead and experiment with:
* Make the network wider
* Make the network deeper
* Add L2 regularization
 * https://pytorch.org/docs/stable/generated/torch.optim.SGD.html
* Set initial weights using Glorot initialization
 * https://pytorch.org/docs/stable/nn.init.html?highlight=xavier_normal#torch.nn.init.xavier_normal_
* Add batch normalization
 * https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
* Add dropout
 * https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
* Change ReLU activation
 * https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
* Change the optimizer
 * https://pytorch.org/docs/stable/generated/torch.optim.Adam.html
 
How do these changes affect training and the final accuracy?

# Lecture 2, Part 2: Convolutional Neural Networks

In this part you will learn how to build and train a Convolutional Neural Network (CNN). For this exercise, we will use the dataset containing 872666 25x25 px *jet images* (see arXiv:1701.05927) and corresponding labels: 1 for signal, i.e. W boson and 0 for background, i.e. QCD, from [here](https://zenodo.org/record/269622/#.YtgOpEhBxe7).

Traditional approaches to jet tagging rely on features, such as jet substructure, designed by experts that detect characteristic energy deposit patterns. In recent years, many studies applied computer vision for event reconstruction at particle colliders. This was obtained by projecting the lower level detector measurements of the emanating particles onto a cylindrical detector and then unwrapping the inner surface of the calorimeter on a rectangle. Such information was further interpreted as an image with calorimeter cells as pixels, where pixel intensity maps the energy deposit of the cell, i.e. **jet images**. The different appearance of these jets can be used as a handle to discriminate between them, i.e. **jet tagging**.

## Convolution Operation

Two-dimensional convolutional layer for image height $H$, width $W$, number of input channels $C$, number of output kernels (filters) $N$, and kernel height $J$ and width $K$ is given by:

\begin{align}
\label{convLayer}
\boldsymbol{Y}[v,u,n] &= \boldsymbol{\beta}[n] + \sum_{c=1}^{C} \sum_{j=1}^{J} \sum_{k=1}^{K} \boldsymbol{X}[v+j,u+k,c]\, \boldsymbol{W}[j,k,c,n]\,,
\end{align}

where $Y$ is the output tensor of size $V \times U \times N$, $W$ is the weight tensor of size $J \times K \times C \times N$ and $\beta$ is the bias vector of length $N$ .

The example below uses a sharpening filter with $C=1$, $J=K=3$:

<img src="assets/convolution.gif"/>[Credit](https://towardsdatascience.com/types-of-convolution-kernels-simplified-f040cb307c37)

Let's write our own convolution operation and see how does it perform:

In [ ]:
def convolution(img, kernel, stride=None, padding=None):
    
    H = img.shape[0]
    W = img.shape[1]
    
    # Let's assume S=J=K
    S = kernel.shape[0]
    filt_h = kernel.shape[1]
    
    # In our case we only have one filer, so n=1. We will drop the 3rd dim for simplicity
    img_out = np.zeros((H+1,W+1))

    # Nested loops over V and U
    for v in range(S//2, H-S//2):
        for u in range(S//2, W-S//2):
            img_out[v, u] = np.sum(img[v-S//2:v+S//2+1,u-S//2:u+S//2+1]*kernel)
    return img_out

In [ ]:
# Load and show an image

import matplotlib.image as mpimg

img = mpimg.imread('assets/cms.png')
plt.imshow(img);

# This is the CMS detecor BTW

A lot of filters were hand-crafted a long time ago. Apps on your phone are often still using these man-made kernes. We are going to to use an classic edge detection filter. You can read more about it [here](https://en.wikipedia.org/wiki/Sobel_operator).

In [ ]:
# Define our filter

sobel_filter = np.array([[1,2,1], [0,0,0], [-1, -2, -1]])
sobel_filter

In [ ]:
# Perform our convolution and show the result

img_edges = convolution(img[:, :, 0], sobel_filter)
plt.imshow(img_edges, cmap='binary');

The edges should be white or black colored (depending on the direction of change), gray areas mean no edge. We have used a vertical variant of the sobel filter. We can see the edges quite clear. Let's try to see if there is any difference when we apply a horizontal version.

In [ ]:
# Perform our convolution and show the result

img_edges = convolution(img[:, :, 0], sobel_filter.T)
plt.imshow(img_edges, cmap='binary');

Let's do a check if our input image matches the shape of the output one:

In [ ]:
img[:,:,0].shape == img_edges.shape

Oh... But that's expected. Do you know why?

## Stride and padding

There are two important parameters that we did not implement, i.e. stride and padding.

**Stride** contols by how many pixels do we move in every loop. Image from [here](https://www.oreilly.com/library/view/hands-on-transfer-learning/9781788831307/df581a77-c057-4978-a25e-564cb1ab5bb3.xhtml).

![](./assets/stride.png)

**Padding** expands the input by adding border to the image, e.g:

In [ ]:
A = np.random.random((4,4))
print(A)
print(np.pad(A, 1, constant_values=0))

### Your task: Implement a stride and padding in the convolution function

## Pooling Operation

Pooling layers are used to reduce the dimensions of the feature maps, simultaneously reducting number of learnable parameters and speeding forward and backward pass. The operation slides a filter over the feature map which either outputs and average or max.

In [ ]:
def pooling(img, size):
    # Mind that I used size as stride as in preactis they are often equal. As an optional exercise you can correct this method, i.e `def pooling(img, size, stride)`
    
    H = img.shape[0]
    W = img.shape[1]

    img_out = np.zeros((H//size,W//size))

    # Nested loops over V and U
    for v in range(0, H//size):
        for u in range(0, W//size):
            img_out[v, u] = np.max(img[v*size:v*size+size,u*size:u*size+size])
    return img_out

In [ ]:
# Perform our pooling and show the result

img_pool = pooling(img[:, :, 0], 4)
plt.imshow(img_pool);

Let's verify that the dimensionality indeed decreased:

In [ ]:
img_pool.shape

### Your task: change max pooling to average pooling

## Constructing the CNN

In [ ]:
# Let's build owr networks

import torch.nn.init as init


class LeNet5Tagger(torch.nn.Module):
    def __init__(self):
        super(LeNet5Tagger, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, 5, padding=2)
        self.pool1 = torch.nn.AvgPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.pool2 = torch.nn.AvgPool2d(2, 2)
        self.fc1 = torch.nn.Linear(16*4*4, 120)
        self.fc2 = torch.nn.Linear(120, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)
    
class FullyConnectedTagger(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(FullyConnectedTagger, self).__init__()
        self.fc1 = torch.nn.Linear(in_features, 64, bias=True)
        self.fc2 = torch.nn.Linear(64, 64, bias=True)
        self.fc3 = torch.nn.Linear(64, 32, bias=True)
        self.fc4 = torch.nn.Linear(32, out_features, bias=True)

    def forward(self, x):
        x = x.view(-1, 25*25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=-1)
        return x
    
fcn = FullyConnectedTagger(25*25, 2)
cnn = LeNet5Tagger()

## Downloading the dataset

We are going to download the dataset from [here](https://zenodo.org/record/269622/#.YtgEwUhBxe7). This can take some time...

In [ ]:
!wget -q 'https://zenodo.org/record/269622/files/jet-images_Mass60-100_pT250-300_R1.25_Pix25.hdf5?download=1' -O './jet_images.h5'

In [ ]:
# Read the file that we just downloaded

import h5py
dataset = h5py.File('jet_images.h5', 'r')

In [ ]:
# Let's see how does the W boson looks like

average_w = np.mean(dataset['image'][:100000][dataset['signal'][:100000] == 1], axis=0)
plt.imshow(average_w);

In [ ]:
# And here is an average of QCD

average_qcd = np.mean(dataset['image'][:100000][dataset['signal'][:100000] == 0], axis=0)
plt.imshow(average_qcd);

In [ ]:
batch_size = 25

class H5Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = torch.Tensor(images)
        self.labels = torch.Tensor(labels)

    def __getitem__(self, index):
        return self.images[index], self.labels[index]

    def __len__(self):
        return len(self.labels)
    
train_dataloader = torch.utils.data.DataLoader(
    H5Dataset(
        torch.Tensor(dataset['image'][:100000]).unsqueeze(1),
        torch.LongTensor(dataset['signal'][:100000])
    ),
    batch_size=batch_size,
    num_workers=0,
    shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    H5Dataset(
        torch.Tensor(dataset['image'][100000:150000]).unsqueeze(1),
        torch.LongTensor(dataset['signal'][100000:150000])
    ),
    batch_size=batch_size,
    num_workers=0,
    shuffle=True
)


## Training and evaluating the models

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
epochs = 10

acc, loss = AverageMeter('Accuracy'), AverageMeter('Loss')

# Iterate over the dataset <epochs> times
for epoch in range(epochs):

    # Set the model to training mode
    cnn.train()
    # Reset our meters
    loss.reset()
    acc.reset()

    # This is just here because it's pretty 
    tr = trange(len(train_dataloader), file=sys.stdout)

    # Iterate over batches
    for inputs, targets in train_dataloader:

        # Remove previous gradients
        optimizer.zero_grad()
        
        # Feed forward the input
        outputs = cnn(inputs)

        # Compute the loss and accuracy
        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)
        
        preds = torch.argmax(outputs, dim=-1)

        accuracy = (targets == preds).sum() / len(targets)
        acc.update(accuracy.data)

        # Show the current results
        tr.set_description('Epoch {}, {}, {}'.format(epoch+1, loss, acc))
        tr.update(1)

        # Compute the gradients
        loss_batch.backward()

        # Update parameters
        optimizer.step()
    
    # Validation for each epoch
    cnn.eval()
    loss.reset()
    acc.reset()

    tr = trange(len(val_dataloader), file=sys.stdout)

    for inputs, targets in val_dataloader:

        outputs = cnn(inputs)

        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)

        preds = torch.argmax(outputs, dim=-1)
        accuracy = (targets == preds).sum() / len(targets)
        acc.update(accuracy.data)

        tr.set_description('Validation, {}, {}'.format(loss, acc))
        tr.update(1)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn.parameters(), lr=0.001)
epochs = 10

acc, loss = AverageMeter('Accuracy'), AverageMeter('Loss')

# Iterate over the dataset <epochs> times
for epoch in range(epochs):

    # Set the model to training mode
    fcn.train()
    # Reset our meters
    loss.reset()
    acc.reset()

    # This is just here because it's pretty 
    tr = trange(len(train_dataloader), file=sys.stdout)

    # Iterate over batches
    for inputs, targets in train_dataloader:

        # Remove previous gradients
        optimizer.zero_grad()
        
        # Feed forward the input
        outputs = fcn(inputs)

        # Compute the loss and accuracy
        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)
        
        preds = torch.argmax(outputs, dim=-1)

        accuracy = (targets == preds).sum() / len(targets)
        acc.update(accuracy.data)

        # Show the current results
        tr.set_description('Epoch {}, {}, {}'.format(epoch+1, loss, acc))
        tr.update(1)

        # Compute the gradients
        loss_batch.backward()

        # Update parameters
        optimizer.step()
    
    # Validation for each epoch
    fcn.eval()
    loss.reset()
    acc.reset()

    tr = trange(len(val_dataloader), file=sys.stdout)

    for inputs, targets in val_dataloader:

        outputs = fcn(inputs)

        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)

        preds = torch.argmax(outputs, dim=-1)
        accuracy = (targets == preds).sum() / len(targets)
        acc.update(accuracy.data)

        tr.set_description('Validation, {}, {}'.format(loss, acc))
        tr.update(1)

In [ ]:
# Let's check the accuracy on the test data

from sklearn.metrics import accuracy_score

test_images = torch.Tensor(dataset['image'][200000:250000]).unsqueeze(1)
y_truth = dataset['signal'][200000:250000]
y_pred_cnn = cnn(test_images)
y_pred_fcn = fcn(test_images)

print("Accuracy for the CNN: {0:.2f}".format(
    accuracy_score(
        y_truth,
        torch.argmax(y_pred_cnn, dim=-1).squeeze().numpy())
))

print("Accuracy for the fully connected: {0:.2f}".format(
    accuracy_score(
        y_truth,
        torch.argmax(y_pred_fcn, dim=-1).squeeze().numpy())
))

### Your task: visualize ROC curves and compute the AUC for both networks.

CNN is a little better than fully connected network. But we use much less parameters:

In [ ]:
# Check the size

print("Numer of parameters in CNN: {}".format(sum(p.numel() for p in cnn.parameters())))
print("Numer of parameters in FCN: {}".format(sum(p.numel() for p in fcn.parameters())))

Go ahead and experiment with:
* Make the network wider (more filters)
* Make the network deeper
* Change the size of the kernel
* Change to average pooling
* Add batch normalization

How do these changes affect training and the final accuracy?

## Transfer learning

This is the last part for today. With transfer learning we can apply knowledge learned on one task to a new one. Why? Depending on the size deep neural networks require hours, days or weeks to train. We can cut this time by applying pre-trained weights used for image recognition. Let's do that:

In [ ]:
from torchvision.models import mobilenet_v2

def weights_init(m):
    init.kaiming_uniform_(m.weight.data)

# Load the pretrained model from torchvision
model = mobilenet_v2(pretrained=True)

# # Replace the first layer to match the input
model.features[0][0] = torch.nn.Conv2d(1,
                              32,
                              kernel_size=3,
                              stride=2,
                              padding=(1,1),
                              bias=False)
model.features[0][0].apply(weights_init)

# # Replace the layer to match the input
model.classifier[1] = torch.nn.Linear(1280, 2, bias=True)
model.classifier[1].apply(weights_init)
print(model)

We will use MobileNetv2. You can read more about it [here](https://ai.googleblog.com/2018/04/mobilenetv2-next-generation-of-on.html).

In [ ]:
# Retrain. It can be a bit intense for a CPU, but let's try

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 2

acc, loss = AverageMeter('Accuracy'), AverageMeter('Loss')

# Iterate over the dataset <epochs> times
for epoch in range(epochs):

    # Set the model to training mode
    model.train()
    # Reset our meters
    loss.reset()
    acc.reset()

    # This is just here because it's pretty 
    tr = trange(len(train_dataloader), file=sys.stdout)

    # Iterate over batches
    for inputs, targets in train_dataloader:

        # Remove previous gradients
        optimizer.zero_grad()
        
        # Feed forward the input
        outputs = model(inputs)

        # Compute the loss and accuracy
        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)
        
        preds = torch.argmax(outputs, dim=-1)

        accuracy = (targets == preds).sum() / len(targets)
        acc.update(accuracy.data)

        # Show the current results
        tr.set_description('Epoch {}, {}, {}'.format(epoch+1, loss, acc))
        tr.update(1)

        # Compute the gradients
        loss_batch.backward()

        # Update parameters
        optimizer.step()
    
    # Validation for each epoch
    model.eval()
    loss.reset()
    acc.reset()

    tr = trange(len(val_dataloader), file=sys.stdout)

    for inputs, targets in val_dataloader:

        outputs = model(inputs)

        loss_batch = criterion(outputs, targets)
        loss.update(loss_batch.data)

        preds = torch.argmax(outputs, dim=-1)
        accuracy = (targets == preds).sum() / len(targets)
        acc.update(accuracy.data)

        tr.set_description('Validation, {}, {}'.format(loss, acc))
        tr.update(1)

As the last exercise, would you like to try a different architecure? [Here](https://pytorch.org/vision/0.11/models.html), check the documentation. Mind that some networks will be too big to train a CPU.